In [ ]:
import pandas as pd
import pysolr

In [ ]:
url = "http://b2find.eudat.eu/solr"
# url = "http://localhost:8983/solr/ckan"

solr = pysolr.Solr(url)

In [ ]:
facet = "extras_spatial"

solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
    'facet': 'true',
    'facet.limit': 100000,
    'facet.mincount': 1,
    'facet.sort': 'count',
    'facet.field': [facet]
}

In [ ]:
results = solr.search(**solr_params)

In [ ]:
# results.facets['facet_fields'][facet]

In [ ]:
result_counts = results.facets["facet_fields"][facet]
geom = result_counts[0::2]
counts = result_counts[1::2]

In [ ]:
import geopandas
from shapely.geometry import shape
import json

In [ ]:
geometry = [shape(json.loads(i)).buffer(1) for i in geom]
gdf = geopandas.GeoDataFrame(list(zip(geometry, counts)), columns =['geometry', 'counts'], geometry="geometry")
gdf


In [ ]:
gdf['centroid'] = gdf.centroid
gdf['x'] = gdf.centroid.x
gdf['y'] = gdf.centroid.y
gdf

In [ ]:
import datashader as ds
import colorcet as cc
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(gdf, x="x", y="y", agg=ds.sum("counts"))
img = ds.tf.shade(agg, cmap=cc.fire, how="log")
img

In [ ]:
import holoviews as hv
from holoviews.element.tiles import OSM, StamenTerrain
from cartopy import crs
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

df = pd.DataFrame()
df["x"] = gdf.x*100000
df["y"] = gdf.y*140000

map_tiles  = OSM().opts(width=850, height=500, alpha=0.7, projection=crs.WGS84_SEMIMAJOR_AXIS)
points     = hv.Points(df, kdims=["x", "y"])
locations = datashade(points, x_sampling=1, y_sampling=1, cmap=cc.fire, width=850, height=500, 
                      cnorm="log", 
                      alpha=200,
                     )

map_tiles * locations